In [1]:
import pandas as pd
import numpy as np
import os.path

from keras.preprocessing.image import Iterator
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

import threading
lock = threading.Lock()

from keras.callbacks import ModelCheckpoint

from BsonIteratorChannelfirstOrder import BSONIterator

In [2]:
METADATA_DIR = 'metadata'
DATA_DIR = '../../dataset'

train_offsets_df = pd.read_csv(os.path.join(METADATA_DIR,"train_offsets.csv"), index_col=0)
train_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Train_0.5_0.1.csv"), index_col=0)
val_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Val_0.5_0.1.csv"), index_col=0)

num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 16

# Create a generator for training and a generator for validation.
# Tip: use ImageDataGenerator for data augmentation and preprocessing.
assert os.path.exists(os.path.join(DATA_DIR, 'train.bson'))
train_bson_file = open(os.path.join(DATA_DIR, 'train.bson'), "rb")
train_datagen = ImageDataGenerator()
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df,
                         num_classes, train_datagen, lock, target_size = (224,224),
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator()
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock, target_size = (224,224),
                       batch_size=batch_size, shuffle=True)

/home/ankdesh/virtualenvs/cdiscount-py2.7/local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Found 5570369 images belonging to 5270 classes.
Found 614245 images belonging to 5270 classes.


In [3]:
asd = train_gen.next()

In [4]:
print (asd[0].shape)

(16, 224, 224, 3)


In [14]:
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.models import Model,load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications import mobilenet
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint

import ResnetFactory

# Save model Dir
MODEL_DIR = 'saved_models'

# Checkpointer 
checkpointer = ModelCheckpoint(filepath=MODEL_DIR+'/Resnet101.{epoch:02d}-{acc:.2f}-{val_acc:.2f}_RandomSplit_Train_0.5_0.1.h5', verbose=1)

model = ResnetFactory.ResnetBuilder.build_resnet_101((3,224,224),5270)

In [15]:
# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
opt = Adam(lr=0.001)
model.compile(optimizer=opt,
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [16]:
num_epochs = 2


# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4, callbacks=[checkpointer])

Epoch 1/2
   999/348148 [..............................] - ETA: 168830s - loss: 11.1703 - acc: 0.0121

KeyboardInterrupt: 

In [7]:
for i,layer in enumerate(model.layers):
    print (i,layer.name)

(0, 'input_1')
(1, 'conv2d_1')
(2, 'batch_normalization_1')
(3, 'activation_1')
(4, 'max_pooling2d_1')
(5, 'conv2d_2')
(6, 'batch_normalization_2')
(7, 'activation_2')
(8, 'conv2d_3')
(9, 'batch_normalization_3')
(10, 'activation_3')
(11, 'conv2d_5')
(12, 'conv2d_4')
(13, 'add_1')
(14, 'batch_normalization_4')
(15, 'activation_4')
(16, 'conv2d_6')
(17, 'batch_normalization_5')
(18, 'activation_5')
(19, 'conv2d_7')
(20, 'batch_normalization_6')
(21, 'activation_6')
(22, 'conv2d_8')
(23, 'add_2')
(24, 'batch_normalization_7')
(25, 'activation_7')
(26, 'conv2d_9')
(27, 'batch_normalization_8')
(28, 'activation_8')
(29, 'conv2d_10')
(30, 'batch_normalization_9')
(31, 'activation_9')
(32, 'conv2d_11')
(33, 'add_3')
(34, 'batch_normalization_10')
(35, 'activation_10')
(36, 'conv2d_12')
(37, 'batch_normalization_11')
(38, 'activation_11')
(39, 'conv2d_13')
(40, 'batch_normalization_12')
(41, 'activation_12')
(42, 'conv2d_15')
(43, 'conv2d_14')
(44, 'add_4')
(45, 'batch_normalization_13')
(46,

In [19]:
model.save('saved_models/Mobilenet_finetune.2denselayer.01-0.44-0.45_RandomSplit_Train_0.5_0.1.h5')